In [ ]:
!pip3 install SimpleITK
!pip install numpyencoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from pathlib import Path

import plotly.express as px

import os
import json
import random
from tqdm import tqdm

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import SimpleITK as sitk

import skimage
from skimage.io import imread, imshow, imsave
from skimage.transform import resize
from skimage.util import *
from scipy import ndimage
from skimage import measure
import plotly.express as px
import copy

from numpyencoder import NumpyEncoder

current_dir = '/content/drive/MyDrive/Colab Notebooks/Kidney Cancer Challenge'
os.chdir(current_dir)
from utils import load_case
from visualize import visualize

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Caricamento dizionari

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Kidney Cancer Challenge/train_info'
fp= open(file_path)
volumes_train_info = json.loads(fp.read())

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Kidney Cancer Challenge/val_info'
fp= open(file_path)
volumes_val_info = json.loads(fp.read())

### Funzioni

In [ ]:
def create_path(gen_path, folder_name):
  # Funzione che riceve in input il nome di una cartella da creare e restituisce il percorso 
  # appena generato; se la cartella indicata è già presente all'interno del drive viene 
  # restituito il percorso (già esistente) relativo a questa
  # args:   - gen_path: percorso 'genitore' all'interno del quale viene creata la cartella folder_name
  #         - folder_name: nome della cartella da creare (o da indirizzare, se già presente nel drive)
  # output: - path: percorso relativo alla cartella folder_name
  path = os.path.join(gen_path, folder_name)
  if not os.path.exists(path):
      os.mkdir(path)   
  return path

In [ ]:
def to_original_spacing(data, volumes_train_info, n):  
# Funzione per ripristinare la risoluzione originale quando si effettua la conversione da array a immagine. 
# args:     - data: volume 
#           - volmes_train_info: dizionario
#           - n: indice del soggetto
# out:      - array convertito in immagine
    original_spacing = volumes_train_info["{}".format(n)]['original_spacing']
    original_size = volumes_train_info["{}".format(n)]['original_size']

    data_itk = data_to_sitk_image(data, spacing = volumes_train_info["{}".format(n)]['spacing_2D'])
    data_itk = resample_image(data_itk, out_spacing = original_spacing, out_size = original_size,  is_label = True, is_prediction=True)
  
    return sitk_image_to_data(data_itk)

In [ ]:
def resample_image(image, out_spacing=(1.0, 1.0, 1.0), out_size=None, is_label=False, pad_value=0, is_prediction= False, is_3D = False):
    """Resamples an image to given element spacing and output size."""

    original_spacing = np.array(image.GetSpacing())
    original_size = np.array(image.GetSize())

    if len(out_spacing)==2:
      out_spacing = (original_spacing[0], out_spacing[0], out_spacing[1])

    if out_size is None:
        out_size = np.round(np.array(original_size * original_spacing / np.array(out_spacing))).astype(int)
    else:
      if len(out_size)==2:
        out_size = np.array(out_size)
        out_size_temp = np.round(np.array(original_size * original_spacing / np.array(out_spacing))).astype(int)
        out_size = np.insert(out_size,0,out_size_temp[0])
      elif len(out_size)==3:
        out_size = np.array(out_size)
    
    original_direction = np.array(image.GetDirection()).reshape(len(original_spacing),-1)
    original_center = (np.array(original_size, dtype=float) - 1.0) / 2.0 * original_spacing
    out_center = (np.array(out_size, dtype=float) - 1.0) / 2.0 * np.array(out_spacing)

    original_center = np.matmul(original_direction, original_center)
    out_center = np.matmul(original_direction, out_center)
    out_origin = np.array(image.GetOrigin()) + (original_center - out_center)

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size.tolist())
    resample.SetOutputDirection(image.GetDirection())
    resample.SetOutputOrigin(out_origin.tolist())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(pad_value)

    if is_label:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        #resample.SetInterpolator(sitk.sitkBSpline)
        resample.SetInterpolator(sitk.sitkLinear)

    if is_prediction:
        return resample.Execute(sitk.Cast(image, sitk.sitkFloat32))
    elif is_3D:
        return resample.Execute(image)
    else:
        return resample.Execute(sitk.Cast(image, sitk.sitkUInt8))  

In [ ]:
def sitk_image_to_data(image):
    data = sitk.GetArrayFromImage(image)
    if len(data.shape) == 3:
      data = np.rot90(data, 1, axes=(0, 2))     # rotazione con cui è stata allenata la rete
    return data

def data_to_sitk_image(data, spacing=(1., 1., 1.),is_2D = False):
    if len(data.shape) == 3:
      data = np.rot90(data, -1, axes=(0, 2))     # rotazione con cui è stata allenata la rete
    image = sitk.GetImageFromArray(data)
    image.SetSpacing(np.asarray(spacing, dtype= float))
    return image

In [ ]:
def display2(img1, img2, title1='Image 1', title2='Image 2', color_bar = True): 
  fig = plt.figure(figsize=(20, 10))
  ax1 = plt.subplot(1, 3, 1)
  z1_plot = ax1.imshow(img1, cmap=plt.cm.gray)
  ax1.set_title(title1)
  if color_bar:
   fig.colorbar(z1_plot, ax=ax1)

  ax2 = plt.subplot(1, 3, 2)
  z2_plot = ax2.imshow(img2, cmap=plt.cm.gray)
  ax2.set_title(title2)
  if color_bar:
    fig.colorbar(z2_plot, ax=ax2)

def display_slicer(mask_3d):
  print(f'Slice dimension...: ', mask_3d.shape[1], 'x', mask_3d.shape[2],'\nSlice number......: ', mask_3d.shape[0])
  fig = px.imshow(mask_3d[:,:,:], animation_frame=0, binary_string=True, labels=dict(animation_frame='slice'))
  fig.show()

In [ ]:
def cut_selection(mask_auto, n, volumes_info): 
  # - la funzione label del modulo measure esegue la connected component analysis;
  #   dandole in input un volume ci restituisce una maschera delle stesse dimensioni 
  #   di questo, al cui interno viene associata una label diversa ad ogni regione individuata  
  #   NB: con connectivity=3 diciamo di essere interessati all'individuazione di 
  #       componenti connessi nelle 3 dimensioni (quindi volumi)
  # - la funzione regionprops restituisce le proprietà di ciascuna delle regioni individuate
  aree = []             # vettore delle aree delle regioni - ci sarà utile per individuare le due aree maggiori
  labels = []           # vettore che conterrà le label che sono state associate a ciascuna regione individuata
  centroids = []        # vettore dei centroidi delle regioni; questa informazione ci servirà alla fine di tutto, quando dovremo ricostruire le dimensioni iniziali avremo bisogno di sapere dove 'ancorare' le maschere automatiche fornite in output dalla 3D
  bboxes = []
  mask_labels = measure.label(mask_auto, connectivity = 3)
  regions = measure.regionprops(label_image=mask_labels)
  for region in regions:
    aree.append(region.area)
    labels.append(region.label)
    centroids.append(region.centroid)
    bboxes.append(region.bbox)

  # Hp: le regioni di interesse sono quelle con estensione maggiore
  # nota: potremmo perdere qualche tumore separato dal rene
  area1 = max(aree)
  ind1 = aree.index(area1)
  label1 = labels[ind1]
  centroid1 = np.round(np.array(centroids[ind1]))

  bbox1 = bboxes[ind1]
  volumes_info['{}'.format(n)]['centroid_1_out'] = list(centroid1)     # qui prima c'era "Centroide 1 manuale"
  aree.pop(ind1)
  labels.pop(ind1)
  centroids.pop(ind1)
  
  if (len(aree)!=0) and (max(aree)>area1/25):    # condizione per considerare solo aree 'significative'  
    area2 = max(aree)
    ind2 = aree.index(area2)
    label2 = labels[ind2]
    centroid2 = np.round(np.array(centroids[ind2]))
    if centroid2[2] in range(bbox1[2],bbox1[5]):
      # Non considerare più questa regione
      aree.pop(ind2)
      labels.pop(ind2)
      centroids.pop(ind2)
      # cerca la successiva 
      area2 = max(aree)
      ind2 = aree.index(area2)
      label2 = labels[ind2]
      centroid2 = np.round(np.array(centroids[ind2]))

    volumes_info['{}'.format(n)]['centroid_2_out'] = list(centroid2)        # qui prima c'era "Centroide 2 manuale"
    # selezione della coordinata (x) di 'taglio'
    for region in regions:
      if region.label == label1:
        x1_min = region.bbox[2]
        x1_max = region.bbox[5]
        # print('bbox1: ', region.bbox)
      elif region.label == label2:
        x2_min = region.bbox[2]
        x2_max = region.bbox[5]
        # print('bbox2: ', region.bbox)
    if x1_max > x2_max:    
      # la regione 1 si trova a destra
      if x1_min < x2_max:  # se le due regioni sono parzialmente sovrapposte...
        #...scegliamo come punto di taglio il punto medio della sovrapposizione
        cut = x2_max + ((x2_max-x1_min)/2)    
      else:
        #...altrimenti il punto medio dello spazio "vuoto" tra le due regioni
        cut = x2_max + (x1_min-x2_max)/2      
    else:             
      # la regione 2 si trova a destra
      if x2_min < x1_max:
        cut = x1_max + ((x1_max-x2_min)/2)
      else:
        cut = x1_max + (x2_min-x1_max)/2
  else:
    # se è presente un solo grande componente connesso (e.g. caso 5), allora non viene effettuato alcun taglio 
    cut = 0      

  volumes_info['{}'.format(n)]['cut'] = int(cut)
  return int(cut), volumes_info

def volume_separation(volume, cut):
  volume1 = volume[:,:,:cut]
  volume2 = volume[:,:,cut:]
  return volume1, volume2

In [ ]:
def get_centroid(subvolume):
  aree = []             # vettore delle aree delle regioni - ci sarà utile per individuare le due aree maggiori
  centroids = []        # vettore dei centroidi delle regioni
  
  mask_labels = measure.label(subvolume, connectivity = 3)
  regions = measure.regionprops(label_image=mask_labels)
  for region in regions:
    aree.append(region.area)
    centroids.append(region.centroid)
  area1 = max(aree)
  ind1 = aree.index(area1)
  centroid = np.round(np.array(centroids[ind1])).astype(int)
  return centroid

In [ ]:
def get_ROI(volume, centroid, ROI_shape):
    # ROI_shape deve essere un tuple contenente le dimensioni della ROI (e.g. (128,128,128))
    VD,VH,VW = volume.shape
    RD,RH,RW = ROI_shape
    cz,cy,cx=centroid

    if cx-RW//2 < 0:
      w_min = 0
      ROI_w_min = RW//2 - cx
    else:
      w_min = cx-RW//2
      ROI_w_min = 0
    if cx+RW//2 > VW-1:
      w_max = VW-1
      ROI_w_max = (RW//2)+VW-1-cx
    else:
      w_max = cx+RW//2
      ROI_w_max = RW 

    if cy-RH//2 < 0:
      h_min = 0
      ROI_h_min = RH//2 - cy
    else:
      h_min = cy-RH//2
      ROI_h_min = 0
    if cy+RH//2 > VH-1:
      h_max = VH-1
      ROI_h_max = (RH//2)+VH-1-cy
    else:
      h_max = cy+RH//2
      ROI_h_max = RH 

    if cz-RD//2 < 0:
      d_min = 0
      ROI_d_min = RD//2 - cz
    else:
      d_min = cz-RD//2
      ROI_d_min = 0
    if cz+RD//2 > VD-1:
      d_max = VD-1
      ROI_d_max = (RD//2)+VD-1-cz
    else:
      d_max = cz+RD//2
      ROI_d_max = RD      

    ROI = np.zeros(ROI_shape)
    ROI[ROI_d_min:ROI_d_max,ROI_h_min:ROI_h_max,ROI_w_min:ROI_w_max] = volume[d_min:d_max,h_min:h_max,w_min:w_max]
      
    return ROI

In [ ]:
def get_bbox(segmentation):
    aree = []
    mask_labels = measure.label(segmentation, connectivity = 3)
    regions = measure.regionprops(label_image=mask_labels)
    for region in regions:
      aree.append(region.area)
    ind1 = aree.index(max(aree))
    region = regions[ind1]
    return region.bbox

def check_bbox(vol_auto_half, n, VOI_shape):
  bbox = get_bbox(vol_auto_half)
  d = bbox[3]-bbox[0]
  h = bbox[4]-bbox[1]
  w = bbox[5]-bbox[2] 

  # Scommetare sotto solo nella fase di preparazione all'allenamento della 3D
  # if d>VOI_shape[0] and (h>VOI_shape[1] or w>VOI_shape[2]) :
  #   print(f'\nCaso {n}: la bounding-box ha dimensioni superiori alla VOI - ({d},{h},{w})')
  # elif d>VOI_shape[0]: 
  #   print(f'\nCaso {n}: la bounding-box ha dimensioni superiori alla VOI - ({d},{h},{w})')
  # elif (h>VOI_shape[1] or w>VOI_shape[2]):
  #   print(f'\nCaso {n}: la bounding-box ha dimensioni superiori alla VOI - ({d},{h},{w})')
  return bbox

In [ ]:
def alignment(SEG, MASK, centroid):
  # descrizione: Questa funzione serve a posizionare il risultato della rete 3D all'interno 
  #              di un volume di segmentazione che servirà a ricostruire il volume di partenza 
  # args:        - SEG: 3d array inizializzato a zero al cui interno verrà posizionata MASK; []
  #              - MASK: maschera automatica output della 3D, viene posizionata all'interno di SEG facendo coincidere 
  #                      il suo centroide con il centroide originale del volume a cui si riferisce
  #              - centroid: centroide nel SR di SEG, quindi riferito al volume ricostruito nel postprocess della 2D
  # funzioni:    get_centroid(), get_bbox()

  # coordinate del centroide di riferimento (SR di SEG)
  z_cs, y_cs, x_cs = centroid
  # print("centroide salvato: {}".format(centroid))
  z_cs = int(np.ceil(z_cs))
  y_cs = int(np.ceil(y_cs))
  x_cs = int(np.ceil(x_cs))
  # estrazione del centroide dalla maschera (SR di MASK)
  MASK_bin = copy.deepcopy(MASK)
  MASK_bin[MASK_bin>1]=1

  if sum(sum(sum(MASK_bin))) != 0:
    C = get_centroid(MASK_bin)
    z_cm, y_cm, x_cm = C
    z_cm = int(np.ceil(z_cm))
    y_cm = int(np.ceil(y_cm))
    x_cm = int(np.ceil(x_cm))
    # estrazione delle coordinate della bounding box che racchiudono solo il volume individuato
    bbox = get_bbox(MASK_bin)
    z_min, y_min, x_min, z_max, y_max, x_max = bbox
    # calcolo degli spostamenti della bounding box rispetto al centroide della maschera automatica (C)
    dx_2 = x_max - x_cm
    dx_1 = x_cm - x_min
    dy_2 = y_max - y_cm
    dy_1 = y_cm - y_min
    dz_2 = z_max - z_cm
    dz_1 = z_cm - z_min
    
    # Per risolvere il problema velocemente porto i negativi a 0
    Z1 = z_cs-dz_1
    Z2 = z_cs+dz_2
    Y1 = y_cs-dy_1
    Y2 = y_cs+dy_2
    X1 = x_cs-dx_1
    X2 = x_cs+dx_2

    if Z1 < 0:
      z_min = z_min + abs(Z1)
      Z1 = np.clip(z_cs-dz_1, 0, 1000)
    elif Y1 < 0:
      y_min = y_min + abs(Y1)
      Y1 = np.clip(y_cs-dy_1, 0, 1000)
    elif X1 <0:
      x_min = x_min + abs(X1)
      X1 = np.clip(x_cs-dx_1, 0, 1000)
    if Z2>SEG.shape[0]:
      z_max = z_max - (Z2-SEG.shape[0])
    if Y2>SEG.shape[1]:
      y_max = y_max - (Y2-SEG.shape[1])
    if X2>SEG.shape[2]:
      x_max = x_max - (X2-SEG.shape[2])

    SEG[Z1:Z2, Y1:Y2, X1:X2] = MASK[z_min:z_max, y_min:y_max, x_min:x_max]

  return SEG

In [ ]:
# CONTROLLO - NUOVA da usare
def preprocess_3D(mask_auto, volume, n, volumes_info, is_val = False, is_test = False ):
  # mask_auto: segmentazione ottenuta dalla 2D (volume ricostruito)
  # volume: segmentazione manuale (per train e val), o volume (per train, val, test)
  
  out_spacing = (1.5, 1.5, 1.5)
  ROI_shape = (128,128,128)
  spacing_2D = volumes_info["{}".format(n)]["spacing_2D"]
  obtained_size_2D = volumes_info["{}".format(n)]["obtained_size_2D"]

  # converto maschera e volume a immagine ITK per ricampionare la risoluzione
  mask_auto_itk = data_to_sitk_image(mask_auto, spacing = spacing_2D)
  volume_itk = data_to_sitk_image(volume, spacing = spacing_2D)
  
  # Ricampiono alla risoluzione di input della 3D
  volume_itk = resample_image(volume_itk, out_spacing=out_spacing, out_size = (384,384), is_label=False, pad_value=0, is_3D = True)
  volume = sitk_image_to_data(volume_itk)
  mask_auto_itk = resample_image(mask_auto_itk, out_spacing=out_spacing, out_size = (384,384), is_label=True, pad_value=0)
  mask_auto = sitk_image_to_data(mask_auto_itk)
  volumes_info["{}".format(n)]["obtained_size_3D"] = mask_auto.shape

  # 1) Individuazione taglio e separazione volumi (individuazione centroidi di riferimento nella maschera che andrà riempita dopo la 3D; salvo tutte le info nel dizionario)
  cut, volumes_info = cut_selection(mask_auto, n, volumes_info)
  if cut == 0:
    bbox_1 = check_bbox(mask_auto, n, ROI_shape)
    volumes_info["{}".format(n)]["bbox_1"] = volumes_info["{}".format(n)]["bbox_2"] =  bbox_1
    centroid_1 = get_centroid(mask_auto) 
    volumes_info["{}".format(n)]["centroid_1_out"] = volumes_info["{}".format(n)]["centroid_2_out"] =  centroid_1
    # Allineamento dei centroidi e cropping della ROI desiderata 
    ROI1_vol = get_ROI(volume,centroid_1,ROI_shape).astype(np.uint8)
    ROI2_vol = copy.deepcopy(ROI1_vol).astype(np.uint8)
  else:
    vol1, vol2 = volume_separation(volume, cut)
    msk1, msk2 = volume_separation(mask_auto, cut)
    # 2) Individuazione bounding box attorno ai due complessi
    bbox_1 = check_bbox(msk1, n, ROI_shape)
    volumes_info["{}".format(n)]["bbox_1"] = bbox_1
    bbox_2 = check_bbox(msk2, n, ROI_shape)
    volumes_info["{}".format(n)]["bbox_2"] = bbox_2
    # 3) Individuo i centroidi dei due reni
    centroid_1 = get_centroid(msk1)
    centroid_2 = get_centroid(msk2)
    # 4) Allineamento dei centroidi e cropping della ROI desiderata 
    ROI1_vol = get_ROI(vol1,centroid_1,ROI_shape).astype(np.uint8)
    ROI2_vol = get_ROI(vol2,centroid_2,ROI_shape).astype(np.uint8)

  return ROI1_vol, ROI2_vol, volumes_info

## Preparazione e salvataggio ROI

In [ ]:
# Cartella contenente i volumi (già preprocessati) da cui estrapolare le ROI
folder = 'Data'
path_train_vol = "/".join((current_dir,folder,'train_volumes'))
path_train_mask = "/".join((current_dir,folder,'train_masks'))
path_val_vol = "/".join((current_dir,folder,'val_volumes'))
path_val_mask = "/".join((current_dir,folder,'val_masks'))

# Cartella contenente le maschere automatiche da cui estrapolare le ROI
path_prediction = current_dir+'/'+'prediction_2D'
path_prediction_train = path_prediction+'/'+'training'
path_prediction_val = path_prediction+'/'+'validation'


In [ ]:
#Creazione delle cartelle che conterranno le ROI estratte
path_ROI = create_path(current_dir, 'ROI')                              # CAMBAIRE NOME DATASET A SECONDA DELLA PROVA
path_ROI_train_vol = create_path(path_ROI,'train_volumes')
path_ROI_train_mask_auto = create_path(path_ROI, 'train_masks_auto')
path_ROI_train_mask = create_path(path_ROI, 'train_masks_manual')
path_ROI_val_vol = create_path(path_ROI, 'val_volumes')
path_ROI_val_mask_auto = create_path(path_ROI, 'val_masks_auto')
path_ROI_val_mask = create_path(path_ROI, 'val_masks_manual')

In [ ]:
# Estrazione della lista dei casi
path_training = current_dir+'/'+'training'
train_vol = sorted(os.listdir(path_training))

for n, id_ in tqdm(enumerate(train_vol), total=len(train_vol)):  
  mask_auto = np.load(path_prediction_train+'/'+id_+'_auto.npy')   # maschera automatica
  mask = np.load(path_train_mask+'/'+id_+'_segmentation.npy')      # maschera manuale
  volume = np.load(path_train_vol+'/'+id_+'_imaging.npy')

  ROI1_mask, ROI2_mask, ROI1_vol, ROI2_vol = preprocess_3D(mask_auto, volume, n, volumes_train_info, is_auto= True)
  ROI1_mask, ROI2_mask, ROI1_vol, ROI2_vol, volumes_train_info = preprocess_3D(mask, volume, n, volumes_train_info)

  np.save("/".join((path_ROI_train_mask_auto,str(id_)+'_ROI_1.npy')), ROI1_mask)
  np.save("/".join((path_ROI_train_mask_auto,str(id_)+'_ROI_2.npy')), ROI2_mask)
  np.save("/".join((path_ROI_train_mask,str(id_)+'_ROI_1.npy')), ROI1_mask)
  np.save("/".join((path_ROI_train_mask,str(id_)+'_ROI_2.npy')), ROI2_mask)
  np.save("/".join((path_ROI_train_vol,str(id_)+'_ROI_1.npy')), ROI1_vol)
  np.save("/".join((path_ROI_train_vol,str(id_)+'_ROI_2.npy')), ROI2_vol)

json.dump(volumes_train_info,open('train_info_128','w'), indent=4, cls=NumpyEncoder)   

# Nota: per far rientrare tutto il rene meglio ROI shape di 128x128x128

100%|██████████| 100/100 [17:32<00:00, 10.53s/it]


In [ ]:
# Estrazione della lista dei casi
path_validation = current_dir+'/'+'validation'
val_vol = sorted(os.listdir(path_validation))

for n, id_ in tqdm(enumerate(val_vol), total=len(val_vol)): 
  mask_auto = np.load(path_prediction_val+'/'+id_+'_auto.npy')
  mask = np.load(path_val_mask+'/'+id_+'_segmentation.npy')
  volume = np.load(path_val_vol+'/'+id_+'_imaging.npy')

  ROI1_mask, ROI2_mask, ROI1_vol, ROI2_vol = preprocess_3D(mask_auto, volume, n+100, volumes_val_info, is_validation= True, is_auto=True)
  ROI1_mask, ROI2_mask, ROI1_vol, ROI2_vol, volumes_val_info = preprocess_3D(mask, volume, n+100, volumes_val_info, is_validation=True)

  np.save("/".join((path_ROI_val_mask_auto,str(id_)+'_ROI_1.npy')), ROI1_mask)
  np.save("/".join((path_ROI_val_mask_auto,str(id_)+'_ROI_2.npy')), ROI2_mask)
  np.save("/".join((path_ROI_val_mask,str(id_)+'_ROI_1.npy')), ROI1_mask)
  np.save("/".join((path_ROI_val_mask,str(id_)+'_ROI_2.npy')), ROI2_mask)
  np.save("/".join((path_ROI_val_vol,str(id_)+'_ROI_1.npy')), ROI1_vol)
  np.save("/".join((path_ROI_val_vol,str(id_)+'_ROI_2.npy')), ROI2_vol)

json.dump(volumes_val_info,open('val_info_128','w'), indent=4, cls=NumpyEncoder)

100%|██████████| 50/50 [08:34<00:00, 10.29s/it]


### Verifica del corretto salvataggio delle ROI

In [ ]:
import random

In [ ]:
# training
n = random.randint(0,100)
# n = 88
id_ = 'case_' + str(n).zfill(5)
print(id_)
mask_manual = mask = np.load(path_train_mask+'/'+id_+'_segmentation.npy')
ROI1_mask = np.load("/".join((path_ROI_train_mask,str(id_)+'_ROI_1.npy')))
ROI2_mask = np.load("/".join((path_ROI_train_mask,str(id_)+'_ROI_2.npy')))

ROI1_vol = np.load("/".join((path_ROI_train_vol,str(id_)+'_ROI_1.npy')))
ROI2_vol = np.load("/".join((path_ROI_train_vol,str(id_)+'_ROI_2.npy')))

In [ ]:
# validation
n = random.randint(100,150)
id_ = 'case_' + str(n).zfill(5)
print(id_)
mask_manual = mask = np.load(path_val_mask+'/'+id_+'_segmentation.npy')
ROI1_mask = np.load("/".join((path_ROI_val_mask,str(id_)+'_ROI_1.npy')))
ROI2_mask = np.load("/".join((path_ROI_val_mask,str(id_)+'_ROI_2.npy')))

ROI1_vol = np.load("/".join((path_ROI_val_vol,str(id_)+'_ROI_1.npy')))
ROI2_vol = np.load("/".join((path_ROI_val_vol,str(id_)+'_ROI_2.npy')))

case_00100
